In [30]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
customers =  pd.read_csv("C:\\Users\\M71846\\Desktop\\customers.csv")
orders = pd.read_csv("C:\\Users\\M71846\\Desktop\\orders.csv")

customers['id'] = customers['id'].astype(str)
customers['id'] = customers['id'].str.strip()
orders['user_id'] = orders['user_id'].astype(str)
orders['user_id'] = orders['user_id'].str.strip()
df = pd.merge(customers,orders , how = 'left',left_on = 'id',right_on = 'user_id')
df = df[['id_x','created_x','order_number','created_y']]
df.rename(columns = {'id_x':'user_id','created_x':'signup_date','created_y':'order_date'},inplace = True)
df.sort_values(['signup_date'], ascending = False).head()
df['signup_date'] = pd.to_datetime(df['signup_date'])
df['order_date'] = pd.to_datetime(df['order_date'])
df['signup_date1'] = pd.DatetimeIndex(df.signup_date).normalize()
df['order_date1'] = pd.DatetimeIndex(df.order_date).normalize()
df = df[['user_id','order_number','signup_date1','order_date1']]
df['diff'] = df['order_date1'].sub(df['signup_date1'], fill_value=0)
df.sort_values(['signup_date1'],ascending = False,inplace = True)
df['diff'] =  df['diff'].dt.days
df['date_range'] = df['diff'].apply(lambda x :calculate_range(x) )
date1 = df[:1]['signup_date1'].tolist()[0]
date2 = df[-2:-1]['signup_date1'].tolist()[0]
reange_date = cohort(date1,date2)
df['cohort'] = df['signup_date1'].apply(lambda x : [y[0].strftime('%Y/%m/%d')+'-' + y[1].strftime('%Y/%m/%d') for y in reange_date if y[0]<=x<=y[1]][0])


In [31]:
def calculate_range(diff):
    if diff < 0:
        return '0'
    if len(str(diff))>1:
        if diff%10 != 0:
            zing = int(str(diff)[0])+1
        else:
            zing = int(str(diff)[0])
    else:
        zing = 1
    week = str(0+7*(zing-1)) +str('-')+str(6+7*(zing-1))
    return week

def  cohort(date1,date2):
    import datetime
    range_list = []
    #range2 = date.strftime('%Y/%m/%d')
    while date1 > date2:
        range2 = date1 - datetime.timedelta(6)
        #range1 = r.strftime('%Y/%m/%d')
        range_list.append((range2,date1))
        date1 = range2 -datetime.timedelta(1)
        #range12 = range1 + '-' +range2
    return range_list

In [32]:
df.fillna('No Order',inplace = True)

In [36]:
df.drop_duplicates(['user_id','date_range'],inplace = True)

In [40]:
df['total_order'] = df['order_number'].apply(lambda x :  1 if not x =='No Order' else 0  )
#df['cumsum'] = df['user_id'].apply(lambda x :  1 if x else 0 )
df['users'] = df['user_id'].apply(lambda x :  1 if x else 0 )

In [42]:
len(df['user_id'].tolist())

32463

In [41]:
len(set(df['user_id'].tolist()))

25716

In [43]:
df['new'] = np.where(df['order_number'] == 1.0,1,0)

In [ ]:
df['cumsum'] = df.groupby('cohort')['cumsum'].transform(pd.Series.cumsum)

In [44]:
df = df[['users','cohort','date_range','new','total_order']]

In [45]:
df.reset_index(inplace = True,drop = True)

In [89]:
cols = df2.columns.tolist()

In [91]:
cols = ['cohort',
 'date_range',
 'users',
 'new',
 'total_order']

In [92]:
df2 = df2[cols]

In [93]:
df2.head()

,cohort,date_range,users,new,total_order
0,2014/12/31-2015/01/06,0,190,0,0
1,2014/12/31-2015/01/06,0-6,32,26,32
2,2014/12/31-2015/01/06,14-20,13,2,13
3,2014/12/31-2015/01/06,21-27,11,1,11
4,2014/12/31-2015/01/06,28-34,8,1,8


In [48]:
df2 = df.groupby(['cohort','date_range']).sum().reset_index()

In [100]:
df_user = df2.groupby(['cohort'])['users'].sum().reset_index()

In [108]:
df_user.sort_values('cohort',inplace = True,ascending = True)

In [107]:
df_user.rename(columns = {'users':'Customers'},inplace = True)

In [110]:
df_user['Customers'] = df_user['Customers'].cumsum()

In [220]:
df5 = pd.merge(df2,df_user,how ='left',on ='cohort')

In [221]:
df5 = df5[df5['date_range'] != '0']

In [222]:
df5.sort_values(['cohort','date_range'],ascending = False,inplace = True)

In [223]:
df5 = df5[['cohort','date_range','Customers','total_order','new']]

In [224]:
df5['total_order1'] = df5['total_order']/df5['Customers']*100

In [225]:
df5['new1'] = df5['new']/df5['Customers']*100

In [226]:
df5 = df5.round(3)

In [227]:
df5['total_order1'] = df5['total_order1'].apply(lambda x : str(x)+'% orderers')
df5['new1'] = df5['new1'].apply(lambda x : str(x)+'% 1st time')

In [228]:
df5['total_order'] = df5['total_order'].apply(lambda x : ' ('+ str(x) + ')\n')
df5['new'] = df5['new'].apply(lambda x : ' ('+ str(x) + ')')

In [229]:
df5['total_order1'] = df5['total_order1'] + df5['total_order']
df5['new1'] = df5['new1'] + df5['new']

In [230]:
df5['abc']  =df5['total_order1']+df5['new1']

In [242]:
df5 = df5[['cohort','Customers','date_range','abc']]

In [ ]:
df5[]

In [238]:
from IPython.display import display, HTML
def pretty_print(df):
  display( HTML( df.to_html().replace("\\n","<br>") ) )
  return df.to_html().replace("\\n","<br>")

In [243]:
#df_new = pretty_print(df5)

In [260]:
df6 = df5.pivot(index='cohort', columns='date_range', values='abc')

In [303]:
df6.columns.name = None

In [304]:
col = ['0-6','7-13', '14-20', '21-27', '28-34', '35-41', '42-48', '49-55', '56-62','63-69' ]

In [305]:
df6 = df6[col]

In [306]:
df7 = df6.reset_index()

In [307]:
df8 = df5[['cohort','Customers']]

In [285]:
#

In [308]:
df9 = pd.merge(df7,df8 ,how = 'left',on = 'cohort')

In [297]:
df9.index.name = None

In [310]:
table_1 = df9.to_html(classes='table',index=False,escape=False)

df9

In [ ]:
df2 = df.groupby(['cohort','date_range']).sum().reset_index()

In [ ]:
df2['users1'] = df2.groupby('cohort')['users'].transform(pd.Series.cumsum)

In [ ]:
df.groupby(['cohort','date_range']).sum().unstack()

In [ ]:
#d
df.pivot(index = 'cohort',columns = 'date_range',values ='total_order')
#df.columns

In [312]:
#table_1

In [313]:
df9

,cohort,0-6,7-13,14-20,21-27,28-34,35-41,42-48,49-55,56-62,63-69,Customers
0,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
1,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
2,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
3,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
4,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
5,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
6,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
7,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1.603% orderers (5)\n0.321% 1st time (1),312
8,2014/12/31-2015/01/06,10.256% orderers (32)\n8.333% 1st time (26),8.013% orderers (25)\n1.282% 1st time (4),4.167% orderers (13)\n0.641% 1st time (2),3.526% orderers (11)\n0.321% 1st time (1),2.564% orderers (8)\n0.321% 1st time (1),2.885% orderers (9)\n0.321% 1st time (1),1.603% orderers (5)\n0.0% 1st time (0),1.282% orderers (4)\n0.0% 1st time (0),3.205% orderers (10)\n0.641% 1st time (2),1